### Time Series Analysis
```
- Advanced Machine Learning, Innopolis University 
- Professor: Muhammad Fahim 
- Teaching Assistant: Gcinizwe Dlamini
```
<hr>

```
Lab Plan
    1. Background
    2. Dataset
    2. Stationarity
    3. Trend
    4. Seasonality
    5. ARIMA

```

<hr>

## 1. Background

1. Why & where Time Series is used?
2. What are the components of Time Series? - (Seasonality, Trend, Noise, Cyclicity)
3. What is Stationarity? over different time periods
    * constant mean.
    * constant variance or standard deviation.
    * Auto-covariance should not depend on time

## 1.2 Dataset

The data is about the number of passengers in an Airline. We want to forecast the number of passengers in the future. The data contains a particular month and number of passengers travelling in that month. The data type here is object (month)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [ ]:
data = pd.read_csv('AirPassengers.csv')

from datetime import datetime
con=data['Month']
data['Month']=pd.to_datetime(data['Month'])
data.set_index('Month', inplace=True)
#check datatype of index

data.head()

In [ ]:
#convert to time series:
ts = data['#Passengers']
ts.head(10)

## 2. Stationarity

Stationarity is an important characteristic of time series. A time series is said to be stationary if its statistical properties do not change over time. In other words, it has constant mean and variance, and covariance is independent of time.


**Dickey-fuller Test** :This is one of the statistical tests for checking stationarity. First we consider the null hypothesis: the time series is non-stationary. The result from the rest will contain the test statistic and critical value for different confidence levels. The idea is to have Test statistics less than critical value, in this case we can reject the null hypothesis and say that this Time series is indeed stationary.


**Why do we care about it?**

1. The more predictable and not changing the data is the easier we can predict it.

If your data is stationary then it should be an easy job to build a good model for predicting it, but in most cases, data will never be stationary.
We test for stationarity by checking the values of Rolling mean and Rolling std. or by using Dickey fuller Test.

In [ ]:
plt.plot(ts)
plt.title("Trend Plot")
plt.xlabel('Date')
plt.ylabel('Number of air passengers')
plt.show()

It’s clear from the plot that there is an overall increase in the trend and with some seasonality in it.

## 2.1 Stationarity testing 

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = None
    rolstd = None

    #Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.xlabel("Time (year)")
    plt.show()
    

    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

In [ ]:
test_stationarity(ts)

This is not stationary because :

* mean is increasing even though the std is small.
* Test stat is > critical value.


There are two major factors that make a time series non-stationary. 

They are:
1. Trend: non-constant mean
2. Seasonality: Variation at specific time-frames

The basic idea is to model the trend and seasonality in this series, so we can remove it and make the series stationary. Then we can go ahead and apply statistical forecasting to the stationary series. And finally we can convert the forecasted values into original by applying the trend and seasonality constrains back to those that we previously separated.

## 3. TREND 

The first step is to reduce the trend using transformation, as we can see here that there is a strong positive trend. 

These transformation can be log, sq-rt, cube root etc . 

Basically it penalizes larger values more than the smaller. 
In this case we will use the logarithmic transformation.

In [ ]:
ts_log = np.log(ts)
plt.plot(ts_log)
plt.title("Trend Plot")
plt.xlabel("Time")
plt.ylabel("# of passengers \n (Log scale)")
plt.show()

Check the y axis it is definetly less intense but still the trend is there.

## 3.1 Smoothing 

In smoothing we usually take the past few instances (rolling estimates) We will discuss two methods under smoothing- Moving average and Exponentially weighted moving average.




## 3.2 Moving average
First take $x$ consecutive values and this depends on the frequency if it is 1 year we take 12 values. Lucky for us that Pandas has a function for rolling estimate 

In [ ]:
moving_avg = ts_log.rolling(12).mean()
plt.plot(ts_log)
plt.plot(moving_avg, color='red')

In [ ]:
ts_log_moving_avg_diff = ts_log - moving_avg
ts_log_moving_avg_diff.head(12)

The reason there are null values is because we take the average of first 12 so 11 values are null. We can also see that in the visual representation. Thus it is dropped for further analysis. Now let’s parse it to the function to check for stationarity.


In [ ]:
ts_log_moving_avg_diff.dropna(inplace=True)
ts_log_moving_avg_diff.head()

In [ ]:
test_stationarity(ts_log_moving_avg_diff)

We notice two things:

* The rolling values are varying slightly but there is no specific trend.
* The test statistics is smaller than the 5 % critical values. That tells us that we are 95% confident that this series is stationary.


In this example we can easily take a time period (12 months for a year), but there are situations where the time period range is more complex like stock price etc. So we use the exponentially weighted moving average (there are other weighted moving averages but for starters, lets use this). The previous values are assigned with a decay factor. Pandas again comes to the rescue with some awesome functions for it, like:

## 3.3 Exponentially weighted moving average

In [ ]:
expwighted_avg = ts_log.ewm(halflife=12).mean()
plt.plot(ts_log)
plt.plot(expwighted_avg, color='red')

In [ ]:
ts_log_ewma_diff = ts_log - expwighted_avg
test_stationarity(ts_log_ewma_diff)

It is stationary because:
* Rolling values have less variations in mean and standard deviation in magnitude.
* the test statistic is smaller than 1% of the critical value. So we can say we are almost 99% confident that this is stationary.

## 4. SEASONALITY (ALONG WITH TREND) 

Previously we saw just trend part of the time series, now we will see both trend and seasonality. 

Most Time series have trends along with seasonality. There are two common methods to remove trend and seasonality, they are:
1. Differencing: by taking difference using time lag
2. Decomposition: model both trend and seasonality, then remove them

## 4.1 Differencing

Here we first take the difference of the value at a particular time with that of the previous time. Now let’s do it in Pandas.

In [ ]:
#Take first difference:
ts_log_diff = ts_log - ts_log.shift(1)
plt.plot(ts_log_diff)

In [ ]:
ts_log_diff.dropna(inplace=True)
test_stationarity(ts_log_diff)

It is stationary because:

* the mean and std variations have small variations with time.
* test statistic is less than 10% of the critical values, so we can be 90 % confident that this is stationary.

## 4.2 Decomposing: 

Here we model both the trend and the seasonality, then the remaining part of the time series is returned. Guess what? Yup, we have some awesome function for it. Let’s check it out:

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(ts_log)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.subplot(411)
plt.plot(ts_log, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()

Remove the trend and seasonality from the Time series and now we can use the residual values. Let’s check stationarity.


In [ ]:
ts_log_decompose = residual
ts_log_decompose.dropna(inplace=True)
test_stationarity(ts_log_decompose)

This is stationary because:
1. test statistic is lower than 1% critical values.
2. the mean and std variations have small variations with time.

## 5. FORECASTING A TIME SERIES 

Now that we have made the Time series stationary, let’s make models on the time series using differencing because it is easy to add the error , trend and seasonality back into predicted values .

We will use statistical modelling method called ARIMA to forecast the data where there are dependencies in the values.

Auto Regressive Integrated Moving Average(ARIMA) — It is like a liner regression equation where the predictors depend on parameters (p,d,q) of the ARIMA model.

* p : This is the number of AR (Auto-Regressive) terms. Example — if p is 3 the predictor for y(t) will be y(t-1),y(t-2),y(t-3).

* q : This is the number of MA (Moving-Average) terms.

* d :This is the number of differences or the number of non-seasonal differences .

Now let’s check out on how we can figure out what value of p and q to use. We use two popular plotting techniques; they are:

1. Autocorrelation Function (ACF): It just measures the correlation between two consecutive (lagged version). example at lag 4, ACF will compare series at time instance t1…t2 with series at instance t1–4…t2–4

2. Partial Autocorrelation Function (PACF): is used to measure the degree of association between y(t) and y(t-p).

## 5.1 ACF & PACF 

ACF explains how the present value of a given time series is correlated with the past values 

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARIMA
#ACF and PACF plots:
from statsmodels.tsa.stattools import acf, pacf  

lag_acf = acf(ts_log_diff, nlags=12)
lag_pacf = pacf(ts_log_diff, nlags=12, method='ols')

#Plot ACF:    
plt.subplot(121)    
plt.plot(lag_acf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts_log_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts_log_diff)),linestyle='--',color='gray')
plt.title('Autocorrelation Function')

#Plot PACF:
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts_log_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts_log_diff)),linestyle='--',color='gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

The two dotted lines on either sides of 0 are the confidence intervals. 
These can be used to determine the ‘p’ and ‘q’ values as:

* p: The first time where the PACF crosses the upper confidence interval, here its close to 2. hence p = 2.
* q: The first time where the ACF crosses the upper confidence interval, here its close to 2. hence p = 2.

Now, using this make 3 different ARIMA models considering individual as well as combined effects. I will also print the RSS for each.

The “residuals” in a time series model are what is left over after fitting a model. The residuals are equal to the difference between the observations and the corresponding fitted values:

Please note that here RSS is for the values of residuals and not actual series.

In [ ]:
model = ARIMA(ts_log, order=(2,1,0))
results_AR = model.fit(disp=-1)
plt.plot(ts_log_diff)
plt.plot(results_AR.fittedvalues, color='red')
plt.title('RSS: %.4f'%sum((results_AR.fittedvalues - ts_log_diff)**2))
plt.xlabel("Time (year)")
plt.show()

In [ ]:
model = ARIMA(ts_log, order=(0,1,2))
results_MA = model.fit(disp=-1)
plt.plot(ts_log_diff)
plt.plot(results_MA.fittedvalues, color='red')
plt.title('RSS: %.4f'%sum((results_MA.fittedvalues - ts_log_diff)**2))
plt.xlabel("Time (year)")
plt.show()

In [ ]:
model = ARIMA(ts_log, order=(2,1,2))
results_ARIMA = model.fit(disp=-1)
plt.plot(ts_log_diff)
plt.plot(results_ARIMA.fittedvalues, color='red')
plt.title('RSS: %.4f'%sum((results_ARIMA.fittedvalues - ts_log_diff)**2))
plt.xlabel("Time (year)")
plt.show()

RSS values:
* AR=1.5023
* MA=1.472
* ARIMA =1.0292

ARIMA has the best RSS values.

## 5.3 FINAL STEP: BRINGING THIS BACK TO THE ORIGINAL SCALE

Steps involved:

• First get the predicted values and store it as series. You will notice the first month is missing because we took a lag of 1(shift).

• Now convert differencing to log scale: find the cumulative sum and add it to a new series with a base value (here the first-month value of the log series).

In [ ]:
predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues,copy=True)
predictions_ARIMA_diff.head()

In [ ]:
predictions_ARIMA_diff_cum_sum = predictions_ARIMA_diff.cumsum()
predictions_ARIMA_diff_cum_sum.head()

In [ ]:
predictions_ARIMA_log = pd.Series(ts_log.iloc[0],index=ts_log.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cum_sum,fill_value=0)
predictions_ARIMA_log.head()

• Next -take the exponent of the series from above (anti-log) which will be the predicted value — the time series forecast model.
Now plot the predicted values with the original.
• Find the RMSE

In [ ]:
predictions_ARIMA = np.exp(predictions_ARIMA_log)
plt.plot(ts)
plt.plot(predictions_ARIMA)
plt.title('RMSE: %.4f'%np.sqrt(sum((predictions_ARIMA-ts)**2)))
plt.ylabel("# of passengers")
plt.xlabel("Time (year)")
plt.show()

## References

1. [The Complete Guide to Time Series Analysis and Forecasting](https://towardsdatascience.com/the-complete-guide-to-time-series-analysis-and-forecasting-70d476bfe775)

2. [statsmodels Documentation](https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima_model.ARIMA.html)

3. [Time Series Forecast : A basic introduction using Python.](https://medium.com/@stallonejacob/time-series-forecast-a-basic-introduction-using-python-414fcb963000)